In [34]:
# %% Section 1: Import Libraries and Load Utilities from Files

import os
import sys
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
import pickle
import importlib.util

# Ensure reproducibility
torch.manual_seed(42)

# Step 1: Set the root directory correctly
project_root = os.getcwd()

# Debug: Print current working directory to confirm it's set correctly
print(f"Current working directory set to: {project_root}")

# Define the utils path relative to the current directory
utils_path = os.path.join(project_root, "utils")

# Debug: Print the utils path
print(f"Looking for 'utils' folder at: {utils_path}")

# Function to load a module by its file path
def load_module(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

# Load encoder.py
encoder_path = os.path.join(utils_path, "encoder.py")

# Debug: Print the encoder path
print(f"Looking for 'encoder.py' at: {encoder_path}")

try:
    encoder_module = load_module("encoder", encoder_path)
    Encoder = encoder_module.Encoder
    print(f"Encoder loaded successfully from {encoder_path}")
except FileNotFoundError as e:
    print(f"Error: {e}. The encoder file path is incorrect. Please verify the file exists.")
    raise

# Load decoder.py
decoder_path = os.path.join(utils_path, "decoder.py")
try:
    decoder_module = load_module("decoder", decoder_path)
    DecoderWithAttention = decoder_module.DecoderWithAttention
    print(f"Decoder loaded successfully from {decoder_path}")
except FileNotFoundError as e:
    print(f"Error: {e}. The decoder file path is incorrect.")
    raise

# Load seq2seq.py
seq2seq_path = os.path.join(utils_path, "seq2seq.py")
try:
    seq2seq_module = load_module("seq2seq", seq2seq_path)
    Seq2SeqWithAttention = seq2seq_module.Seq2SeqWithAttention
    print(f"Seq2Seq model loaded successfully from {seq2seq_path}")
except FileNotFoundError as e:
    print(f"Error: {e}. The Seq2Seq file path is incorrect.")
    raise

# %% Section 2: Load Saved Components

# Paths to saved models and data
models_path = os.path.join(project_root, "models")
encoder_weights_path = os.path.join(models_path, "encoder.pth")
decoder_weights_path = os.path.join(models_path, "decoder_with_attention.pth")
seq2seq_weights_path = os.path.join(models_path, "seq2seq_attention_best_model.pth")
embedding_matrix_path = os.path.join(models_path, "embedding_matrix.pth")
word2idx_path = os.path.join(models_path, "word2idx.pkl")

# Load word2idx from word2idx.pkl
try:
    with open(word2idx_path, 'rb') as f:
        word2idx = pickle.load(f)
    print("word2idx loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading word2idx: {e}. Make sure the file exists at {word2idx_path}.")
    raise

# Load embedding matrix
try:
    embedding_matrix = torch.load(embedding_matrix_path, map_location=torch.device('cpu'), weights_only=True)
    print("Embedding matrix loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading embedding matrix: {e}. Make sure the file exists at {embedding_matrix_path}.")
    raise

# Hyperparameters
input_size = len(word2idx)
output_size = len(word2idx)
hidden_size = 256  # Adjusted according to training

# Initialize the models
encoder = Encoder(input_size, embedding_matrix, hidden_size).cpu()
decoder = DecoderWithAttention(output_size, embedding_matrix, hidden_size).cpu()
model = Seq2SeqWithAttention(encoder, decoder, device='cpu').cpu()

# Load pre-trained weights
try:
    encoder.load_state_dict(torch.load(encoder_weights_path, map_location=torch.device('cpu'), weights_only=True))
    decoder.load_state_dict(torch.load(decoder_weights_path, map_location=torch.device('cpu'), weights_only=True))
    model.load_state_dict(torch.load(seq2seq_weights_path, map_location=torch.device('cpu'), weights_only=True))
    print("Model weights loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading model weights: {e}. Make sure the model files exist at {models_path}.")
    raise

encoder.eval()
decoder.eval()
model.eval()

print("Models loaded successfully.")


Current working directory set to: C:\Users\Girija\Documents\GitHub\NLP-Driven-Rasa-Assiste-ChatBot360
Looking for 'utils' folder at: C:\Users\Girija\Documents\GitHub\NLP-Driven-Rasa-Assiste-ChatBot360\utils
Looking for 'encoder.py' at: C:\Users\Girija\Documents\GitHub\NLP-Driven-Rasa-Assiste-ChatBot360\utils\encoder.py
Encoder loaded successfully from C:\Users\Girija\Documents\GitHub\NLP-Driven-Rasa-Assiste-ChatBot360\utils\encoder.py
Decoder loaded successfully from C:\Users\Girija\Documents\GitHub\NLP-Driven-Rasa-Assiste-ChatBot360\utils\decoder.py
Seq2Seq model loaded successfully from C:\Users\Girija\Documents\GitHub\NLP-Driven-Rasa-Assiste-ChatBot360\utils\seq2seq.py
word2idx loaded successfully.
Embedding matrix loaded successfully.
Model weights loaded successfully.
Models loaded successfully.


In [36]:

# %% Section 3: Define Tokenization and Preprocessing Functions

def preprocess_input(text, word2idx, max_len=20):
    """
    Preprocess input text for the Seq2Seq model.
    Args:
        text (str): The input text.
        word2idx (dict): Word to index mapping.
        max_len (int): Maximum length of the token sequence.
    Returns:
        torch.Tensor: The tokenized and padded input sequence.
    """
    tokens = word_tokenize(text.lower())
    sequence = [word2idx.get(token, word2idx['<UNK>']) for token in tokens]
    sequence = [word2idx['<SOS>']] + sequence + [word2idx['<EOS>']]
    sequence = sequence[:max_len] + [word2idx['<PAD>']] * (max_len - len(sequence))
    return torch.tensor(sequence, dtype=torch.long).unsqueeze(0)  # Add batch dimension


In [41]:
def generate_response(input_text, model, word2idx, max_len=20):
    """
    Generate a response from the trained Seq2Seq model.
    Args:
        input_text (str): The input query.
        model (Seq2SeqWithAttention): Trained Seq2Seq model.
        word2idx (dict): Word to index mapping.
        max_len (int): Maximum length of the response sequence.
    Returns:
        str: Generated response.
    """
    try:
        # Tokenize and preprocess the input
        input_tensor = preprocess_input(input_text, word2idx, max_len=max_len)
        input_tensor = input_tensor.to('cpu')

        print(f"Preprocessed input tensor: {input_tensor}")

        # Prepare a placeholder target tensor (start with <SOS> token)
        target_tensor = torch.tensor([word2idx['<SOS>']], dtype=torch.long).unsqueeze(0).to('cpu')

        with torch.no_grad():
            # Forward pass through the Seq2Seq model
            output = model(input_tensor, target_tensor, teacher_forcing_ratio=0.0)

        # Debugging: print output shape and values
        print(f"Model output tensor shape: {output.shape}")
        print(f"Model output tensor: {output}")

        # Extract the predicted tokens
        predicted_indices = output.argmax(2).squeeze().tolist()

        # Ensure `predicted_indices` is a list
        if isinstance(predicted_indices, int):
            predicted_indices = [predicted_indices]

        # Correcting the response generation logic:
        response_tokens = []
        for idx in predicted_indices:
            # Retrieve the token corresponding to the index
            token = next((key for key, value in word2idx.items() if value == idx), None)
            # Append the token to the response if it's not a special token
            if token and token not in ['<SOS>', '<EOS>', '<PAD>']:
                response_tokens.append(token)

        # Debugging: print response tokens
        print(f"Response tokens: {response_tokens}")

        # Reconstruct response text
        response = ' '.join(response_tokens)
        return response

    except Exception as e:
        print(f"Error during response generation: {e}.")
        return ""


In [42]:

# %% Section 5: Testing the Model with a Predefined Input

# Example predefined input
predefined_input = "Hello, how can I assist you today?"

# Generate response
try:
    response = generate_response(predefined_input, model, word2idx)
    print(f"Input: {predefined_input}")
    print(f"Response: {response}")
except Exception as e:
    print(f"Error during response generation: {e}.")

Preprocessed input tensor: tensor([[400001,  13075,      1,    197,     86,     41,   4281,     81,    373,
            188, 400002, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
         400000, 400000]])
Model output tensor shape: torch.Size([1, 1, 400004])
Model output tensor: tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]])
Response tokens: ['the']
Input: Hello, how can I assist you today?
Response: the


In [43]:
# %% Section 5: Testing the Model with a Predefined Set of Inputs

# Predefined set of inputs to test the model
predefined_inputs = [
    "Hello, how can I assist you today?",
    "What is your name?",
    "Can you tell me a joke?",
    "How is the weather today?",
    "What time is it now?"
]

# Generate responses for each predefined input
for predefined_input in predefined_inputs:
    try:
        response = generate_response(predefined_input, model, word2idx)
        print(f"Input: {predefined_input}")
        print(f"Response: {response}\n")
    except Exception as e:
        print(f"Error during response generation for input '{predefined_input}': {e}.")


Preprocessed input tensor: tensor([[400001,  13075,      1,    197,     86,     41,   4281,     81,    373,
            188, 400002, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
         400000, 400000]])
Model output tensor shape: torch.Size([1, 1, 400004])
Model output tensor: tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]])
Response tokens: ['the']
Input: Hello, how can I assist you today?
Response: the

Preprocessed input tensor: tensor([[400001,    102,     14,    392,    311,    188, 400002, 400000, 400000,
         400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000,
         400000, 400000]])
Model output tensor shape: torch.Size([1, 1, 400004])
Model output tensor: tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]])
Response tokens: ['the']
Input: What is your name?
Response: the

Preprocessed input tensor: tensor([[400001,     86,     81,   1361,    285,      7,   7351,    188, 400002,
         400000, 400000, 400000, 400000, 400000, 400000, 400000, 400000, 400